In [28]:
import requests
import json
import os
import subprocess
import sys
import base64
import hashlib
from urllib.parse import urlparse
import subprocess
import json
import time
import os
import glob


RSA_SIZE = 2048


DEFAULT_CURVE = "secp384r1"
FAST_CURVE = "secp256r1"
SUPPORTED_CURVES = [DEFAULT_CURVE, FAST_CURVE]

DIGEST_SHA384 = "sha384"
DIGEST_SHA256 = "sha256"

RSA_SIZE = 2048
# CCF network node
server="https://127.0.0.1:8000"

num_users = 4
# Getting Network metrices
url = server + "/app/api/metrics"

try:
    response = requests.get(url, verify='./workspace/sandbox_common/service_cert.pem')

    print("Status Code:", response.status_code)
    print("\nResponse Headers:")
    for header, value in response.headers.items():
        print(f"{header}: {value}")

    print("\nResponse Body:")
    try:
        # Attempt to parse JSON and print it in an indented format
        response_json = response.json()
        print(json.dumps(response_json, indent=4))
    except ValueError:
        # If response is not JSON, print as plain text
        print(response.text)

except requests.exceptions.RequestException as e:
    print("Error making request:", e)

Status Code: 200

Response Headers:
content-length: 176
content-type: application/json

Response Body:
{
    "metrics": [
        {
            "calls": 1,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "api/metrics",
            "retries": 0
        },
        {
            "calls": 2,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "commit",
            "retries": 0
        }
    ]
}


In [9]:
%pip install matplotlib
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 68.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 100.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.6/479.6 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 96.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 81.5 MB/s eta 0:00:00:00:0100:01
   

In [29]:
import numpy as np
import requests
import json
import base64
import time
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical

server = "https://127.0.0.1:8000"  # Replace with your server URL

def create_model():
    print("Initializing the global model...")
    model = Sequential([
        Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, kernel_size=3, activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def aggregate_weights(model_id,round_no, user_cert, user_key):
    print("Aggregating weights for model:", model_id)
    payload = {"modelId": model_id,"roundNo":round_no}
    response = requests.post(
        url=f"{server}/app/model/aggerate",
        verify="./workspace/sandbox_common/service_cert.pem",
        cert=(user_cert, user_key),
        json=payload
    )
    if response.status_code == 200:
        print("Aggregation successful for model:", model_id)
    else:
        raise Exception(f"Failed to aggregate weights. Status code: {response.status_code}")
def train_model(model, X_train, y_train, X_test, y_test, user_id, round_no):
    print(f"Training model for User {user_id}, Round {round_no}...")
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2)
    return history.history['loss']

def serialize_model(model):
    print("Serializing the model...")
    model.save('temp_model.h5')
    with open('temp_model.h5', 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')
    

def upload_initial_model(model_base64, user_cert, user_key):
    print("Uploading initial global model...")
    payload = {
      "global_model": {
        "model_name": "CNNModel",
        "model_data": model_base64
      }
    }
    response = requests.post(
        url=f"{server}/app/model",
        verify="./workspace/sandbox_common/service_cert.pem",
        cert=(user_cert, user_key),
        json=payload
    )

    if response.status_code == 200:
        model_data = response.json()
        model_id = model_data.get("model_id")
        model_name = model_data.get("model_name")
        print(f"Initial global model '{model_name}' (ID: {model_id}) uploaded successfully.")
        return model_id
    else:
        print(f"Failed to upload initial model. Status code: {response.status_code}")
        return None

def serialize_weights(model):
    """ Serialize only the weights of the model """
    print("Serializing model weights...")
    weights = model.get_weights()  # Get the model weights as a list of numpy arrays
    weights_base64 = base64.b64encode(json.dumps([w.tolist() for w in weights]).encode()).decode()
    return weights_base64

def upload_model_weights(model_weights_base64, user_cert, user_key, round_no, model_id=None):
    """ Upload only the model weights """
    print(f"Uploading model weights for Round {round_no}...")
    payload = {
        "modelId": model_id,
        "weights": model_weights_base64,
        "roundNo": round_no
    }
    response = requests.post(
        url=f"{server}/app/weights",
        verify="./workspace/sandbox_common/service_cert.pem",
        cert=(user_cert, user_key),
        json=payload
    )
    print(response)
    if response:
        print(f"Model weights uploaded successfully for Round {round_no}.")
    else:
        raise Exception(f"Failed to upload model weights. Status code: {response.status_code}")

def download_model(user_cert, user_key, user_id, round_no, max_retries=5, model_id=None):
    attempts = 0
    while attempts < max_retries:
        print(f"Attempting to download model for User {user_id}, Round {round_no}, Attempt {attempts + 1}...")
        response = requests.get(
            url=f"{server}/app/model?model_id={model_id}",
            verify="./workspace/sandbox_common/service_cert.pem",
            cert=(user_cert, user_key)
        )
        print(response.text)
        print(response.status_code)

        if response.status_code == 200:
            model_data = response.json().get("model_details", {})
            model_base64 = model_data
            
            if model_base64:
                with open('temp_model.h5', 'wb') as file:
                    file.write(base64.b64decode(model_base64))
                return load_model('temp_model.h5')
            else:
                print("Model data not found in response, retrying...")
        else:
            print(f"Failed to download model. Status code: {response.status_code}, retrying...")
        
        time.sleep(2)  # Delay before retrying
        attempts += 1
    
    raise Exception("Failed to download model after maximum retries.")

def plot_loss(user_losses):
    for user_id, losses in user_losses.items():
        plt.plot(losses, label=f'User {user_id}')
    plt.title('Model Loss per Training Round')
    plt.ylabel('Loss')
    plt.xlabel('Round')
    plt.legend()
    plt.show()

# Load and preprocess MNIST dataset
print("Loading and preprocessing MNIST dataset...")
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Split the dataset for two users
print("Splitting dataset for two users...")
split_index = int(len(X_train) / 2)
X_train_user0, X_train_user1 = np.split(X_train, [split_index])
y_train_user0, y_train_user1 = np.split(y_train, [split_index])

# Initialize and train the global model (User 0)
global_model = create_model()
train_model(global_model, X_train_user0, y_train_user0, X_test, y_test, user_id=0, round_no=0)

# Serialize and upload initial global model
model_base64 = serialize_model(global_model)
initial_model_id = upload_initial_model(model_base64, "./workspace/sandbox_common/user0_cert.pem", "./workspace/sandbox_common/user0_privk.pem")

if initial_model_id is None:
    raise Exception("Failed to upload the initial global model. Stopping the process.")

# Wait for some time after initial upload
time.sleep(1)  # Wait for 10 seconds

# Federated Learning Process
num_rounds = 4  # Number of training rounds
user_losses = {0: [], 1: []}  # To track losses for each user

for round_no in range(1, num_rounds + 1):
    for user_id in range(2):  # Two users: 0 and 1
        # Download the latest global model
        model = download_model(
            f"./workspace/sandbox_common/user{user_id}_cert.pem",
            f"./workspace/sandbox_common/user{user_id}_privk.pem",
            user_id, round_no, model_id=initial_model_id
        )
        # Train the model on the user's data
        X_train_user = X_train_user0 if user_id == 0 else X_train_user1
        y_train_user = y_train_user0 if user_id == 0 else y_train_user1
        loss = train_model(model, X_train_user, y_train_user, X_test, y_test, user_id, round_no)
        user_losses[user_id].extend(loss)

        # Serialize and upload the updated model weights for the current round
        model_weights_base64 = serialize_weights(model)
        upload_model_weights(
            model_weights_base64,
            f"./workspace/sandbox_common/user{user_id}_cert.pem",
            f"./workspace/sandbox_common/user{user_id}_privk.pem",
            round_no
        )
    aggregate_weights("CNNModel1",round_no,"./workspace/sandbox_common/member0_cert.pem", "./workspace/sandbox_common/member0_privk.pem")
        

print("Federated Learning Process Completed.")

# Plot loss graphs for each user
plot_loss(user_losses)

Loading and preprocessing MNIST dataset...
Splitting dataset for two users...
Initializing the global model...
Training model for User 0, Round 0...
Epoch 1/2
938/938 [==============================] - 13s 13ms/step - loss: 0.6155 - accuracy: 0.9030 - val_loss: 0.1319 - val_accuracy: 0.9572
Epoch 2/2
938/938 [==============================] - 12s 13ms/step - loss: 0.1070 - accuracy: 0.9675 - val_loss: 0.1364 - val_accuracy: 0.9605
Serializing the model...
Uploading initial global model...
Initial global model 'CNNModel' (ID: 0) uploaded successfully.
Attempting to download model for User 0, Round 1, Attempt 1...
{"message":"Model details retrieved successfully","model_details":"...","model_id":0}
200
model_data ...
Training model for User 0, Round 1...
Epoch 1/2
938/938 [==============================] - 13s 13ms/step - loss: 0.0778 - accuracy: 0.9760 - val_loss: 0.1009 - val_accuracy: 0.9728
Epoch 2/2
938/938 [==============================] - 12s 13ms/step - loss: 0.0657 - accuracy: 

Exception: Failed to upload model weights. Status code: 500